In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
path = "/Users/michalmacbookpro/Downloads/titanic"
df_train = pd.read_csv(path + "/train.csv")
df_test = pd.read_csv(path + "/test.csv")
submision = pd.read_csv(path + "/gender_submission.csv")

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, QuantileTransformer

In [51]:
def titulizer(df):
    df["Title"] = df["Name"].str.extract(r"([A-Za-z]+\.)")
    df["Title"].replace(['Capt.', 'Col.', 'Countess.', 'Don.', 'Dona.', 'Dr.', 'Jonkheer.', 'Lady.','Major.', 'Rev.', 'Sir.'], 'Rare.', inplace = True)
    df["Title"].replace(['Mlle.', 'Ms.'], 'Miss.', inplace = True)
    df["Title"].replace(['Mme.'], 'Mrs.', inplace = True)
    return df


#df_train.sample(10)
def cabinizer(df):
    df["Cabin_L"] = df["Cabin"].str.extract(r"(^\D{0,1})")
    df["Cabin_L"].fillna("U",inplace=True)
    df["Cabin_L"] = df["Cabin_L"].map({"A":"ABC","B":"ABC","C":"ABC","D":"DE","E":"DE","F":"FG","G":"FG","T":"ABC","U":"U"})
    return df


def agegrouping(df):
    df["Age_Group"] = pd.cut(df["Age"],5)
    return df

def faregrouping(df):
    df["Fare_Group"] = pd.cut(x=df["Fare"],bins=[0,100,500,1000],labels=["low","middle","high"])
    return df

def age_imp(df):
    df["Age"].fillna(np.mean(df["Age"]),inplace=True)
    return df

def familiazer(df):
    df["Family"] = df["SibSp"] + df["Parch"] + 1
    df["Family"] = df["Family"].astype("str")
    return df

In [5]:
age_imp(df_train)
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df_train.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
140,141,0,3,"Boulos, Mrs. Joseph (Sultana)",female,29.699118,0,2,2678,15.2458,NaN,C
857,858,1,1,"Daly, Mr. Peter Denis",male,51.000000,0,0,113055,26.5500,E17,S
505,506,0,1,"Penasco y Castellana, Mr. Victor de Satode",male,18.000000,1,0,PC 17758,108.9000,C65,C


In [52]:
titulizer(df_train)
cabinizer(df_train)
agegrouping(df_train)
faregrouping(df_train)
familiazer(df_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Cabin_L,Age_Group,Fare_Group,Family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,Mr.,U,"(16.336, 32.252]",low,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,Mrs.,ABC,"(32.252, 48.168]",low,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,U,"(16.336, 32.252]",low,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,Mrs.,ABC,"(32.252, 48.168]",low,2
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,Mr.,U,"(32.252, 48.168]",low,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,Rare.,U,"(16.336, 32.252]",low,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,Miss.,ABC,"(16.336, 32.252]",low,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,Miss.,U,"(16.336, 32.252]",low,4
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,Mr.,ABC,"(16.336, 32.252]",low,1


In [8]:
df_train.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Name             object
Sex              object
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked         object
Title            object
Cabin_L          object
Age_Group      category
Fare_Group     category
Family           object
dtype: object

In [9]:
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [10]:
cat_cols = ["Pclass","Sex","Age_Group","Fare_Group","Title","Embarked"]

In [11]:
cat_pipe = Pipeline([
    ("Encoder",OrdinalEncoder()),
    ("Imputer",SimpleImputer(strategy="most_frequent")),
    ("OneHot",OneHotEncoder(sparse=False))
    ])

In [12]:
preprocesing_pipeline = ColumnTransformer([
    ("cat", cat_pipe,cat_cols)
])

In [13]:
y = df_train["Survived"].copy()

In [14]:
x = df_train[cat_cols]

In [15]:
X = preprocesing_pipeline.fit_transform(x)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier


rfc = RandomForestClassifier()
dtc = DecisionTreeClassifier()

In [41]:
param = [{
    "criterion": ['gini', 'entropy'],
    "n_estimators":[100,200,300,400,500,600],
    'max_depth': range(4, 20, 2),
    'max_leaf_nodes': range(4, 28, 2),
}]

gs_rt = RandomizedSearchCV(rfc, param, cv = 5, n_jobs = -1, verbose = 1)
gs_rt.fit(X_train, y_train)
y_pred = gs_rt.predict(X_test)
#print(gs_rt.best_estimator_)
print(accuracy_score(y_pred,y_test))
rt_rs_best = gs_rt.best_estimator_


Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.8268156424581006


In [53]:
param = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': range(4, 20, 2),
        'max_leaf_nodes': range(4, 28, 2),
    }
]

gs_dt = GridSearchCV(dtc, param, cv = 5, n_jobs = -1, verbose = 1,scoring="accuracy")
gs_dt.fit(X_train, y_train)
y_pred = gs_dt.predict(X_test)
print(accuracy_score(y_pred,y_test))
dt_gs_best = gs_dt.best_estimator_
#print(f1_score(y_pred,y_test))

Fitting 5 folds for each of 192 candidates, totalling 960 fits
0.7877094972067039


In [43]:
from sklearn.svm import SVC

svc = SVC()

param = [
    {
        'kernel': ['poly'], 'C': [0.1, 0.3, 1, 2, 3, 4], 
        'gamma': [0.1,0.2,0.3]
    }, 
]

svc = SVC(probability = True)
rs_svc = RandomizedSearchCV(svc, param, cv = 5, n_jobs = -1, verbose = 1)
rs_svc.fit(X_train, y_train)
y_pred = rs_svc.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(rs_svc.best_estimator_)
print(accuracy_score(y_pred,y_test))
svc_best = rs_svc.best_estimator_


Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.8044692737430168
SVC(C=0.3, gamma=0.2, kernel='poly', probability=True)
0.8044692737430168


In [44]:
param = [
    {'n_estimators': range(8, 28, 4), 
     'max_depth': range(24, 48, 4),
     'max_leaf_nodes': range(20, 48, 4),
    }
]

et = ExtraTreesClassifier()
gs_et = RandomizedSearchCV(et, param, cv = 5, n_jobs = -1, verbose = 1)
gs_et.fit(X_train, y_train)
y_pred = gs_et.predict(X_test)
print(accuracy_score(y_pred,y_test))
gs_et_best = gs_et.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.8212290502793296


In [45]:
param_grid={
    'max_depth': range(2, 10, 2),
    'n_estimators': range(26, 46, 2),
    'learning_rate': [0.2, 0.1, 0.03, 0.01]
}

xg = XGBClassifier(eval_metric='logloss', n_jobs = -1, use_label_encoder = False)
gs_xg = RandomizedSearchCV(xg, param_grid, cv = 5, n_jobs = -1, verbose = 1)
gs_xg.fit(X_train, y_train)
y_pred = gs_xg.predict(X_test)
print(accuracy_score(y_pred,y_test))
gs_xg_best = gs_xg.best_estimator_
print(accuracy_score(y_pred,y_test))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype inste

0.7932960893854749
0.7932960893854749


In [46]:
###eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
#eclf1 = eclf1.fit(X, y)
#print(eclf1.predict(X))###

voting_clf = VotingClassifier(estimators=[
    ("rand_for",rt_rs_best),
    ("dec_tree",dt_gs_best),
    ("svc",svc_best),
    ("extree",gs_et_best),
    ("xgb",gs_xg_best)],voting = 'hard', n_jobs = -1, verbose = 1)
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(accuracy_score(y_pred,y_test))


[Voting] ................. (2 of 5) Processing dec_tree, total=   0.0s
[Voting] ................... (4 of 5) Processing extree, total=   0.0s
[Voting] ...................... (3 of 5) Processing svc, total=   0.1s
[Voting] ...................... (5 of 5) Processing xgb, total=   0.1s
[Voting] ................. (1 of 5) Processing rand_for, total=   0.3s
0.8044692737430168


In [47]:
df_test["Age"].fillna(np.mean(df_test["Age"]),inplace=True)
df_test["Fare"].fillna(np.mean(df_test["Fare"]),inplace=True)
titulizer(df_test)
cabinizer(df_test)
agegrouping(df_test)
faregrouping(df_test)
familiazer(df_test)

#df_test.sample(15)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Cabin_L,Age_Group,Fare_Group,Family
0,892,3,"Kelly, Mr. James",male,34.50000,0,0,330911,7.8292,NaN,Q,Mr.,U,young_adult,low,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00000,1,0,363272,7.0000,NaN,S,Mrs.,U,middle age,low,2
2,894,2,"Myles, Mr. Thomas Francis",male,62.00000,0,0,240276,9.6875,NaN,Q,Mr.,U,middle age,low,1
3,895,3,"Wirz, Mr. Albert",male,27.00000,0,0,315154,8.6625,NaN,S,Mr.,U,young_adult,low,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00000,1,1,3101298,12.2875,NaN,S,Mrs.,U,young_adult,low,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,NaN,S,Mr.,U,young_adult,low,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C,Rare.,ABC,young_adult,middle,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr.,U,young_adult,low,1
416,1308,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,NaN,S,Mr.,U,young_adult,low,1


In [38]:
df_test_test = preprocesing_pipeline.transform(df_test)

In [39]:
submision["Survived"] = voting_clf.predict(df_test_test).astype("int")
submision.to_csv('submission.csv', index = False)

In [48]:
submision.Survived.value_counts()

0    298
1    120
Name: Survived, dtype: int64

In [49]:
submision["Survived"] = gs_et_best.predict(df_test_test).astype("int")
submision.to_csv('submission.csv', index = False)
submision.Survived.value_counts()

0    284
1    134
Name: Survived, dtype: int64